Поскольку MNIST — это своего рода Hello World для современной обработки
изображений, в TensorFlow этот набор данных поддерживается «из коробки»,
и для импорта MNIST достаточно написать буквально две строчки кода:

In [1]:
from tensorflow.examples.tutorials.mnist import input_data 
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

C:\Users\Art\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.


Полученные данные уже разбиты на тренировочную (mnist.train), тестовую
(mnist.test) и валидационную (mnist.validate) выборки, содержащие 55 000, 10 000
и 5000 примеров соответственно. Каждая из этих выборок состоит из изображений
цифр (mnist.train.images) и их меток (mnist.train.labels).

In [2]:
import tensorflow as tf

В TensorFlow требуемые операции выражаются с помощью символьных переменных,
поэтому давайте создадим переменную для тренировочных данных:

In [3]:
x = tf.placeholder(tf.float32, [None, 784])

В данном случае х — это не какой-то заранее заданный тензор, а так называемая
заглушка (placeholder), которую мы заполним, когда попросим TensorFlow произвести
вычисления. Мы хотим иметь возможность использовать произвольное число
784-мерных векторов для обучения, поэтому в качестве одной из размерностей
указываем None. Для TensorFlow это значит, что данная размерность может иметь
произвольную длину.  
Кроме заглушки для тренировочных данных, нам также потребуются переменные,
которые мы собственно и будем изменять при обучении нашей модели.

In [4]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

Здесь W имеет размерность 784 x 10, так как мы собираемся умножать вектор
размерности 784 на W и получать предсказание для 10 возможных меток, а вектор b
размерности 10 — это свободный член, bias, который мы добавляем к выходу.  
После того как мы импортировали нужные модули и объявили все переменные,
собственно нашу модель на TensorFlow можно записать в одну строчку!

In [5]:
y = tf.nn.softmax(tf.matmul(x, W) + b)

Сначала мы перемножаем матрицы х и W с помощью tf.matmul(x,W), затем добавляем к результату Ь, и для получения вероятностей классов применяем tf.nn.softnax.   
Для того чтобы обучить модель, нужно также зафиксировать некий способ оценки качества предсказаний (именно эту оценку мы и будем в конечном счете оптимизировать). Опишем теперь в терминах TensorFlow функцию потерь. Для исходной разметки нам понадобится заглушка:

In [6]:
y_ = tf.placeholder(tf.float32, [None, 10])

И теперь функцию потерь тоже можно записать в одну строчку:

In [7]:
cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

Давайте более подробно разберем эту строку. Посмотрим, что будет происходить, последовательно, от внутренних операций к внешним:
- сначала мы вычисляем tf.log(y), логарифм каждого элемента у;
- затем умножаем каждый элемент у_ на соответствующий ему tf.log(y) (операция умножения на векторах, матрицах и тензорах здесь понимается покомпонентно);
- затем суммируем результат с помощью tf.reduce_sum по второму измерению; напомним, что первое измерение — это примеры из тестового или валидационного множества, а второе — возможные классы, то есть мы суммируем не по примерам, а по размерности каждого вектора у; для этого мы указали в качестве параметра reductlon_lndlces=[1];
- и наконец, последняя операция tf.reduce_mean подсчитывает среднее значение по всем примерам в выборке.

Теперь, когда мы знаем, чего мы хотим от нашей модели, мы можем попросить TensorFlow оптимизировать эту функцию. Да, это именно настолько просто. Мы уже полностью описали граф вычислений в терминах, понятных TensorFlow, все вершины в этом графе содержат известные классические функции, градиенты которых, разумеется, уже реализованы в TensorFlow, и библиотека может воспользоваться алгоритмом обратного распространения ошибки для того, чтобы подсчитать градиенты, узнать, как веса влияют на функцию потерь, которую мы хотим минимизировать, и затем применить тот или иной алгоритм оптимизации для уточнения этих весов.  
Для обучения модели мы будем использовать метод градиентного спуска со скоростью обучения 0,5:

In [8]:
train_step = tf.train.GradientDescentOptimizer(0.5).minimize(cross_entropy)

Перед началом обучения осталось инициализировать переменные:

In [9]:
init = tf.global_variables_initializer()

Теперь мы готовы запускать обучение. Для этого создаем TensorFlow-ceccию...

In [10]:
sess = tf.Session()
sess.run(init)

...и пора запускать:

In [11]:
for i in range(1000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys})

На каждом проходе цикла мы выбираем случайные 100 примеров из обучающей выборки и передаем их в функцию tra in _ s te p для обучения. Такой подход мы уже обсуждали — это типичный стохастический градиентный спуск; в данном случае мы применяем его потому, что обучающая выборка слишком велика для
того, чтобы проходить по ней целиком на каждом шаге обучения (собственно, так будет всегда на любых данных хоть сколько-нибудь реального размера). Вместо этого мы выбираем каждый раз небольшой новый случайный набор обучающих данных и используем его.  
Осталось понять, насколько хорошо мы теперь умеем распознавать рукописные цифры. Модель на данный момент выдает предсказания в виде softmax-результатов — суммирующихся в единицу чисел, отражающих уверенность модели в том или ином ответе. Для того чтобы понять, какую метку мы предсказали для очередного изображения, можно просто взять максимальное значение из этих результатов. В TensorFlow это выражается как tf.argmax, функция, выдающая позицию максимального элемента в тензоре по заданной оси. Для того чтобы понять, верно ли мы предсказали метку, достаточно просто сравнить между собой
**tf.argmax(y, 1)** и **tf.argmax(y_, 1)**:

In [12]:
correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))

Это дает на выходе список булевых значений, показывающих, верно или неверно предсказан результат; итоговой точностью может быть, например, среднее значение соответствующего вектора:

In [13]:
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

Осталось только вычислить его и вывести:

In [14]:
print("Точность: %s" %
      sess.run(accuracy, feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

Точность: 0.9125


=====================================================================================

Итак, первое знакомство с TensorFlow прошло удачно, но в нашем примере не было никаких глубоких сетей, да и вообще нейронных-то сетей по сути нет: мы просто обучили логистическую регрессию.  
Давайте теперь немного усложним модель и посмотрим, что из этого получится.  

Во-первых, чтобы наша модель вообще имела право носить гордое название нейронной сети, нужно добавить в нее скрытый слой, назовём его «слой ReLU». В качестве функции активации возьмем все тот же ReLU. Нужно инициализировать веса и свободный член для этого слоя; будем в этом примере использовать 100 нейронов на скрытом слое:

In [15]:
W_relu = tf.Variable(tf.truncated_normal([784, 100], stddev=0.1))
b_relu = tf.Variable(tf.truncated_normal([100], stddev=0.1))

На этот раз мы инициализируем переменные не нулями, а небольшими случайными значениями. Функция __tf.truncated_normal__ возвращает значения, порожденные нормально распределенной случайной величиной с фиксированными математическим ожиданием (в нашем примере мы оставили значение 0, задающееся по умолчанию) и дисперсией (в нашем примере __stddev=0.l__); однако при этом значения, вышедшие за пределы интервала в **±2σ** от среднего, выбираются заново, то есть распределение обрезается так, чтобы полностью запретить большие выбросы. Инициализация нулями в данном случае была бы совсем бессмысленной, потому что **ReLU(0) = 0**, а значит, при инициализации нулями градиенты совсем не распространялись бы по сети. В нашем же случае примерно половина весов окажется отрицательной, и соответствующие нейроны не будут активироваться вовсе.  

Итак, хотя теория градиентного спуска от этого не меняется никак, на практике хорошая инициализация весов очень важна, чтобы попасть в разумную область значений целевой функции и не застрять в плохом локальном максимуме.   

А сейчас общий вид скрытого слоя получается таким:

In [16]:
h = tf.nn.relu(tf.matmul(x, W_relu) + b_relu)

Применение **tf.nn.relu** тоже будет покомпонентным: фактически мы просто применили функцию **ReLU** к вектору **tf.matmul(x, W_relu) + b_relu**.  

В этой нейронной сети будет очень полезен слой ***дропаута***. Дропаут — это слой, который выбрасывает (обнуляет) выходы некоторых нейронов, выбираемых случайно и заново для каждого обучающего примера. Все, что нам сейчас нужно задать — это вероятность их выбрасывания; для этого мы сначала создадим заглушку:

In [17]:
keep_probability = tf.placeholder(tf.float32)

А дальше TensorFlow, как всегда, всё делает за нас, достаточно только правильно попросить:

In [18]:
h_drop = tf.nn.dropout(h, keep_probability)

Теперь нейроны скрытого слоя будут участвовать в вычислениях с вероятностью **keep_probabllity**, а с вероятностью **1-keep_probability** их выход будет обнулен, и они не будут ни участвовать в предсказании для этого примера, ни обучаться на нем. Поскольку размер внутреннего слоя отличается от входного, нам придется немного поменять параметры внешнего слоя и, кроме того, переписать заключительный softmax-слой:

In [19]:
W = tf.Variable(tf.zeros([100, 10]))
b = tf.Variable(tf.zeros([10]))
y = tf.nn.softmax(tf.matmul(h_drop, W) + b)

При этом наша функция потерь ***cross_entropy*** и оптимизатор ***train_step*** не требуют никаких изменений. А вот вызывать ***sess.run*** нужно с новым параметром ***keep_probability***. Поэтому цикл обучения немного изменился:

In [22]:
for i in range(2000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_probability: 0.5})

Если сделать 2000 шагов обучения нашей новой сети, мы получим:

In [23]:
print("Точность: %s" % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_probability: 1.}))

Точность: 0.9236


==========================================================================================

Чтобы избежать возможного переполнения при слишком большом количестве циклов обучения, создатели TensorFlow знали об этих трудностях и специально подготовили функцию, которая обходит все подводные камни. Давайте использовать ее вместо самописных функций. Зададим промежуточный тензор:

In [31]:
logit = tf.matmul(h_drop, W) + b

Теперь мы не будем сами применять softmax или считать перекрестную энтропию, а воспользуемся готовой функцией:

In [35]:
cross_entropy = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logit, labels=y_))

Больше ничего менять не надо; давайте снова запустим код, теперь уже сделав 10000 шагов обучения, и посмотрим на результат:

In [39]:
for i in range(10000):
    batch_xs, batch_ys = mnist.train.next_batch(100)
    sess.run(train_step, feed_dict={x: batch_xs, y_: batch_ys, keep_probability: 0.5})
    
print("Точность: %s" % sess.run(accuracy, feed_dict={
    x: mnist.test.images, y_: mnist.test.labels, keep_probability: 1.}))

Точность: 0.9241
